In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [44]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [45]:
df = fetch_hourly_rides(12)

2025-03-04 22:27:34,339 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 22:27:34,342 INFO: Initializing external client
2025-03-04 22:27:34,343 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-03-04 22:27:34,987 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.60s) 


In [46]:
df

,pickup_hour,pickup_location_id,rides
0,2025-03-04 20:00:00+00:00,262,30
1,2025-03-04 18:00:00+00:00,68,218
2,2025-03-04 14:00:00+00:00,186,164
3,2025-03-04 12:00:00+00:00,131,0
4,2025-03-04 13:00:00+00:00,154,0
...,...,...,...
4262,2025-03-05 02:00:00+00:00,252,0
4263,2025-03-05 02:00:00+00:00,55,0
4264,2025-03-05 02:00:00+00:00,255,0
4265,2025-03-05 02:00:00+00:00,58,0


In [47]:
df_pred = fetch_predictions(12)

2025-03-04 22:27:46,553 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 22:27:46,556 INFO: Initializing external client
2025-03-04 22:27:46,556 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-03-04 22:27:47,173 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.04s) 


In [48]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,230,215.0,2025-03-05 01:00:00+00:00
1,42,7.0,2025-03-05 01:00:00+00:00
2,99,0.0,2025-03-05 01:00:00+00:00
3,235,0.0,2025-03-05 01:00:00+00:00
4,63,0.0,2025-03-05 01:00:00+00:00
...,...,...,...
748,242,0.0,2025-03-05 03:00:00+00:00
749,144,73.0,2025-03-05 03:00:00+00:00
750,91,1.0,2025-03-05 03:00:00+00:00
751,29,0.0,2025-03-05 03:00:00+00:00


In [49]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [50]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand
0,2025-03-05 01:00:00+00:00,261,1,19.0
1,2025-03-05 01:00:00+00:00,252,0,0.0
2,2025-03-05 01:00:00+00:00,249,36,136.0
3,2025-03-05 01:00:00+00:00,237,7,40.0
4,2025-03-05 01:00:00+00:00,50,5,34.0
...,...,...,...,...
497,2025-03-05 02:00:00+00:00,252,0,0.0
498,2025-03-05 02:00:00+00:00,55,0,1.0
499,2025-03-05 02:00:00+00:00,255,0,1.0
500,2025-03-05 02:00:00+00:00,58,0,0.0


In [51]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [52]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
194,2025-03-05 01:00:00+00:00,2,0,0.0,0.0
461,2025-03-05 02:00:00+00:00,2,0,0.0,0.0
168,2025-03-05 01:00:00+00:00,3,0,0.0,0.0
423,2025-03-05 02:00:00+00:00,3,0,0.0,0.0
96,2025-03-05 01:00:00+00:00,4,3,4.0,1.0
...,...,...,...,...,...
468,2025-03-05 02:00:00+00:00,261,0,30.0,30.0
231,2025-03-05 01:00:00+00:00,262,2,9.0,7.0
352,2025-03-05 02:00:00+00:00,262,2,26.0,24.0
79,2025-03-05 01:00:00+00:00,263,10,128.0,118.0


In [53]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
0,2025-03-05 01:00:00+00:00,261,1,19.0,18.0
1,2025-03-05 01:00:00+00:00,252,0,0.0,0.0
2,2025-03-05 01:00:00+00:00,249,36,136.0,100.0
3,2025-03-05 01:00:00+00:00,237,7,40.0,33.0
4,2025-03-05 01:00:00+00:00,50,5,34.0,29.0
...,...,...,...,...,...
497,2025-03-05 02:00:00+00:00,252,0,0.0,0.0
498,2025-03-05 02:00:00+00:00,55,0,1.0,1.0
499,2025-03-05 02:00:00+00:00,255,0,1.0,1.0
500,2025-03-05 02:00:00+00:00,58,0,0.0,0.0


In [54]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [41]:
mae_by_hour["MAE"].mean()

nan